In [1]:
import model_function as mf

/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:43: DeprecationWarning: This module has been deprecated in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [11]:
import pandas as pd
import psycopg2 as pg2

In [3]:
df = pd.read_csv('/data/npi_2013_clean1.csv')

/home/ubuntu/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df.head()

,Unnamed: 0,npi,last_name,first_name,entity_code,provider_zip,provider_city,specialty_desc,bene_count,total_claim_count,...,lis_claim_cost,nonlis_claim_count,nonlis_claim_cost,brand_claim_cost_avg,generic_claim_cost_avg,other_claim_cost_avg,mapd_claim_cost_avg,pdp_claim_cost_avg,lis_claim_cost_avg,nonlis_claim_cost_avg
0,0,1447268107,10 PARK PHARMACY INC.,NaN,O,100164332,NEW YORK,Pharmacist,12,18,...,0.000000,18.0,885.230000,321.16685,28.298063,35.031374,127.952029,100.548657,154.74375,123.493166
1,355,1952668477,ABANTE,CAROLYN,I,875011456,SANTA FE,Pharmacist,24,24,...,851.090628,5.5,679.212411,321.16685,28.298063,35.031374,127.952029,100.548657,154.74375,123.493166
2,501,1114065679,ABBAS,AGHA,I,104515504,BRONX,Pharmacist,13,26,...,851.090628,5.5,679.212411,321.16685,28.298063,35.031374,127.952029,100.548657,154.74375,123.493166
3,643,1558621474,ABBASPOUR,ABBAS,I,972091029,PORTLAND,Pharmacist,12,12,...,851.090628,5.5,679.212411,321.16685,28.298063,35.031374,127.952029,100.548657,154.74375,123.493166
4,849,1427388651,ABBOTT,KRISTINA,I,850231359,PHOENIX,Pharmacist,25,26,...,851.090628,5.5,679.212411,321.16685,28.298063,35.031374,127.952029,100.548657,154.74375,123.493166


In [5]:
X_train, X_test = mf.make_train_test_dataset(df)

In [19]:
X_train.columns

Index([u'Unnamed: 0', u'npi', u'last_name', u'first_name', u'entity_code',
       u'provider_zip', u'provider_city', u'specialty_desc', u'bene_count',
       u'total_claim_count', u'total_drug_cost', u'total_day_supply',
       u'brand_claim_count', u'brand_claim_cost', u'generic_claim_count',
       u'generic_claim_cost', u'other_claim_count', u'other_claim_cost',
       u'mapd_claim_count', u'mapd_claim_cost', u'pdp_claim_count',
       u'pdp_claim_cost', u'lis_claim_count', u'lis_claim_cost',
       u'nonlis_claim_count', u'nonlis_claim_cost', u'brand_claim_cost_avg',
       u'generic_claim_cost_avg', u'other_claim_cost_avg',
       u'mapd_claim_cost_avg', u'pdp_claim_cost_avg', u'lis_claim_cost_avg',
       u'nonlis_claim_cost_avg'],
      dtype='object')

In [22]:
keep_cols = ['bene_count'
             , 'total_claim_count'
             , 'total_drug_cost'
             , 'total_day_supply'
             , 'brand_claim_count'
             , 'brand_claim_cost'
             , 'generic_claim_count'
             , 'generic_claim_cost'
             , 'other_claim_count'
             , 'other_claim_cost'
             , 'mapd_claim_count'
             , 'mapd_claim_cost'
             , 'pdp_claim_count'
             , 'pdp_claim_cost'
             , 'lis_claim_count'
             , 'lis_claim_cost'
             , 'nonlis_claim_count'
             , 'nonlis_claim_cost'
            ]

In [23]:
mod_if = mf.fit_model(X_train, keep_cols)

[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:   39.6s finished


In [24]:
conn = pg2.connect(dbname='medicare', user='postgres')
query = '''
        SELECT n.npi
            , indict.first_name
            , indict.last_name
            , indict.npi_status
        FROM indictments_2013 indict
            , npi_name n
        WHERE lower(indict.first_name) = lower(n.providerfirstname)
        AND lower(indict.last_name) = lower(n.providerlastnamelegalname)
        AND lower(indict.state) = lower(n.providerbusinessmailingaddressstatename);
        '''

In [25]:
#df_labels = pd.read_sql_query(query, conn)

In [26]:
test_df = mf.check_labels(df_labels, X_test, keep_cols, mod_if)

In [27]:
test_df

,Unnamed: 0,npi,last_name,first_name,entity_code,provider_zip,provider_city,specialty_desc,bene_count,total_claim_count,...,nonlis_claim_cost,brand_claim_cost_avg,generic_claim_cost_avg,other_claim_cost_avg,mapd_claim_cost_avg,pdp_claim_cost_avg,lis_claim_cost_avg,nonlis_claim_cost_avg,prediction,indicted
324221,188072,1619940731,COQUERAN,ADRIENNE,I,235183801,NORFOLK,Family Practice,64.0,742,...,23854.170000,181.149499,22.071589,28.569992,50.773910,58.693464,59.878618,52.614634,0.358304,0
659568,320444,1194831883,GERMER,NANCY,I,801132736,ENGLEWOOD,Obstetrics/Gynecology,31.0,95,...,513.255676,179.648953,34.985739,22.746321,80.178915,87.510205,69.199327,93.319214,0.324329,0
896404,1044827,1679785364,ZETOONY NUDELL,MICHELLE,I,337014408,ST PETERSBURG,Pulmonary Disease,101.0,469,...,65061.280000,363.056301,30.324261,27.547171,179.300939,191.232781,184.029219,191.895298,0.382971,0
382835,770920,1871538967,READ,STEPHEN,I,230594652,GLEN ALLEN,Family Practice,298.0,2828,...,131606.480000,181.149499,22.071589,28.569992,50.773910,58.693464,59.878618,52.614634,0.465467,0
601237,326787,1578506614,GINGRICH,JENNIFER,I,93420,ARROYO GRANDE,Physician Assistant,64.0,121,...,6558.520000,244.991003,23.392842,29.246024,63.094360,73.697529,76.825794,62.633992,0.328915,0
721546,924988,1477998789,TADEVOSYAN,ALEKSEY,I,37561000,LEBANON,Student in an Organized Health Care Education/...,5.5,24,...,1069.390000,240.526093,22.784780,19.180583,55.020564,71.753073,70.360395,56.648265,0.321202,0
875979,572609,1205090958,MAIDA,REBECCA,I,024783607,BELMONT,Optometry,5.5,18,...,498.593287,146.120334,24.401783,26.986278,90.117423,102.051639,107.942792,90.653325,0.323344,0
1036919,998975,1154427185,WEISBERGER,CAROLINE,I,012669236,WEST STOCKBRIDGE,Certified Clinical Nurse Specialist,196.0,4726,...,6258.850000,423.703173,38.692484,41.261574,98.149636,115.266096,118.053218,92.729008,0.552051,0
105020,787401,1073573580,RITTER,STEPHEN,I,329606521,VERO BEACH,Internal Medicine,582.0,5577,...,417436.980000,192.936023,23.082508,29.143358,54.736709,67.127356,67.169662,59.395760,0.554542,0
530171,849331,1194088468,SEVAK,SHRUTI,I,480736712,ROYAL OAK,General Surgery,36.0,59,...,509.840000,214.344588,22.366400,33.475198,50.298379,62.034269,65.986955,51.478635,0.320197,0


In [29]:
test_df[test_df['indicted']==True]

,Unnamed: 0,npi,last_name,first_name,entity_code,provider_zip,provider_city,specialty_desc,bene_count,total_claim_count,...,nonlis_claim_cost,brand_claim_cost_avg,generic_claim_cost_avg,other_claim_cost_avg,mapd_claim_cost_avg,pdp_claim_cost_avg,lis_claim_cost_avg,nonlis_claim_cost_avg,prediction,indicted
459746,333174,1053354787,GOLDFEIN,AARON,I,481242921,DEARBORN,General Practice,497,20657,...,77199.43,162.148990,19.399925,21.653271,39.874235,60.528605,60.136808,42.031533,0.713500,1
13269,40998,1881664829,ATAYA,HATEM,I,486619390,WEST BRANCH,Internal Medicine,868,17664,...,515616.34,192.936023,23.082508,29.143358,54.736709,67.127356,67.169662,59.395760,0.705946,1
526580,689114,1902838014,OGBUOKIRI,GODWIN,I,701263744,NEW ORLEANS,General Surgery,107,1355,...,2330.62,214.344588,22.366400,33.475198,50.298379,62.034269,65.986955,51.478635,0.414670,1
980640,755913,1427136043,QADIR,RIZWAN,I,483141723,STERLING HEIGHTS,Neurology,426,3863,...,260697.01,945.720550,42.924785,23.248386,244.113129,255.598196,273.906044,233.294313,0.534572,1
936622,66443,1497781447,BAZZI,MOHAMAD,I,481873338,CANTON,Plastic and Reconstructive Surgery,64,161,...,34220.89,200.472847,21.124370,24.456168,32.103744,42.045297,52.568557,33.793751,0.380008,1
313717,85833,1427000819,BINDER,WILLIAM,I,481985753,YPSILANTI,Family Practice,73,926,...,4899.50,181.149499,22.071589,28.569992,50.773910,58.693464,59.878618,52.614634,0.361441,1
1030910,206541,1548427164,DANESHVAR,GERALD,I,482123221,HAMTRAMCK,Geriatric Medicine,731,12133,...,104944.78,156.001166,23.273950,28.040818,50.013939,56.604035,54.972580,54.413811,0.631432,1
338511,329079,1528044815,GLAZER,ROBERT,I,905055023,TORRANCE,Family Practice,474,4723,...,282168.83,181.149499,22.071589,28.569992,50.773910,58.693464,59.878618,52.614634,0.547321,1
